In [1]:
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine

In [2]:
check_transactions_df= pd.read_csv('./regex.csv')

In [3]:
check_transactions_df['samplesms'] = check_transactions_df['samplesms'].apply(lambda x: str(x))
check_transactions_df['regex'] = check_transactions_df['regex'].apply(lambda x: str(x))

In [4]:
check_transactions_df.shape

(314, 198)

In [5]:
check_transactions_df = check_transactions_df[~check_transactions_df.regex.isin([np.nan])]

In [6]:
def sure_regex_checks(transactions_df):
#     transactions_df = pd.read_excel(exracted_file)

    transactions_df['Test8'] = transactions_df.apply(lambda x: 'fail' if re.search("[^A-Za-z]+pos[^A-Za-z]+|[^A-Za-z]+ecom[^A-Za-z]+", x['regex'].lower())
                                                                         and not re.search("reverse|refund|credited", x['regex'].lower())
                                                                       and x['smssubtype'] != 'expense' else 'pass', axis = 1)
    print('Test8===completed===========================================================================')

    transactions_df['Test12'] = transactions_df.apply(lambda x: 'fail' if re.search('o/w', x['regex'].lower())
                                                                        and (x['smstype'] != 'credit-transaction' or
                                                                        x['smssubtype'] != 'income' or x['transactiontype'] != 'cheque'
                                                                        or x['transactionchannel'] != 'other') else 'pass', axis = 1)
    print('Test12===completed===========================================================================')

    transactions_df['Test13'] = transactions_df.apply(lambda x: 'fail' if re.search('i/w', x['regex'].lower())
                                                                        and (x['smstype'] != 'debit-transaction' or
                                                                        x['smssubtype'] != 'expense' or x['transactiontype'] != 'cheque'
                                                                        or x['transactionchannel'] != 'other') else 'pass', axis = 1)
    print('Test13===completed===========================================================================')

    # transactions_df['Test15'] = transactions_df.apply(lambda x: 'fail' if re.search('apbs|subsidy', x['regex'].lower())
    #                                                                     and (x['smstype'] != 'credit-transaction'
    #                                                                     or not re.search('income|cash-back', x['smssubtype'].upper())
    #                                                                     or x['transactiontype'] != 'other'
    #                                                                     or x['transactionchannel'] != 'other') else 'pass', axis= 1)
    # print('Test15===completed===========================================================================')

# If ‘IMPS/NEFT/RTGS’ and ‘debited’ in the SMS body then: SMS Type - debit, Account Type- Bank Account, SMS Sub type - Transfer out,  Transaction Channel - Net Banking
    transactions_df['Test16'] = transactions_df.apply(lambda x: 'fail' if re.search('imps|neft|rtgs|p2p|p2a', x['regex'].lower())
                                                                        and re.search('debit|debited', x['regex'].lower())
                                                                         and (x['smstype'] != 'debit-transaction'
                                                                        or x['accounttype'] != 'bank' or x['smssubtype'] != 'transfer-out'
                                                                        or x['transactionchannel'] != 'net-banking')  else 'pass', axis = 1)
    print('Test16===completed===========================================================================')

# If ‘IMPS/NEFT/RTGS’ and ‘credited’ in the SMS body then: SMS Type - credit, Account Type- Bank Account, SMS Sub type - income, Transaction Type - Account Transfer,  Transaction Channel - Net Banking
    transactions_df['Test17'] = transactions_df.apply(lambda x: 'fail' if re.search('imps|neft|rtgs', x['regex'].lower())
                                                                        and re.search('credit|credited', x['regex'].lower())
                                                                        and not re.search('debit|debited', x['regex'].lower())
                                                                       and not re.search('credited to', x['regex'].lower())
                                                               and not re.search('credited to', x['regex'].lower())
                                                                        and (x['smstype'] != 'credit-transaction'
                                                                        or x['accounttype'] != 'bank' or x['transactiontype'] != 'account-transfer'
                                                                        or x['transactionchannel'] != 'net-banking')  else 'pass', axis = 1)
    print('Test17===completed===========================================================================')
      # If smstype is bill and smssubtype is bill-payment-reminder and totalamountdue is -1 or minamountdue then fail otherwise pass
    transactions_df['Test32'] = transactions_df.apply(lambda x: 'fail' if x['smstype'] == 'bill' and x['smssubtype'] == 'bill-payment-reminder' and (x['totalamountdue'] == -1 or x['minamountdue'] == -1 )
                                                                          and x['amount'] != -1 and x['duedate'] == -1 else 'pass', axis = 1)
    print('Test32===completed===========================================================================')

   # If smstype is bill and smssubtype is bill-payment-rejection and transaction amount is -1 then fail otherwise pass
    transactions_df['Test33'] = transactions_df.apply(lambda x: 'fail' if x['smstype'] == 'bill' and x['smssubtype'] == 'bill-payment-rejection' and x['smssubtype'] != 'bill-past-due' else 'pass', axis = 1)
    print('Test33===completed===========================================================================')

   # If smstype is bill and smssubtype is bill-payment-confiramation and transaction amount is -1 then fail otherwise pass
    transactions_df['Test34'] = transactions_df.apply(lambda x: 'fail' if x['smstype'] == 'bill' and x['smssubtype'] == 'bill-payment-confirmation' and x['amount'] == -1 else 'pass', axis = 1)
    print('Test34===completed===========================================================================')

   # If smstype is credit-transaction or debit-transction and transaction amount is -1 and pan is -1 then fail otherwise pass
    transactions_df['Test35'] = transactions_df.apply(lambda x: 'fail' if (x['smstype'] == 'debit-transaction' or x['smstype'] == 'credit-transaction') and x['amount'] == -1
                                                                          and x['pan'] == -1 else 'pass', axis = 1)
    print('Test35===completed===========================================================================')
    
    transactions_df['Test36'] = transactions_df.apply(lambda x: 'fail' if (re.search('imps|neft|rtgs|fund\s*transfer|\s*FT\s*', x['samplesms'].lower())
                                                                       and x['transactionchannel']!='net-banking')
                                                                           else 'pass', axis = 1)
    print('Test36===completed===========================================================================')
    
    transactions_df['Test37'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='credit-transaction' and x['smssubtype']=='cc-bill-payment') and (x['accounttype']!='credit-card') else
                           'pass', axis=1)
    print('Test37===completed===========================================================================')

    transactions_df['Test38'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='credit-transaction' and x['accounttype']=='bank') 
                                                              and (x['smstype'] in ('cc-bill-payment','emi-installment'))else 'pass', axis=1)

    print('Test38===completed===========================================================================')

    transactions_df['Test39'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='credit-transaction' and x['transactiontype']=='account-transfer') 
                                                              and (x['receiveraccountid']!=-1)else 'pass', axis=1)

    print('Test39===completed===========================================================================')

    transactions_df['Test40'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='credit-transaction') 
                                                              and (x['minamountdue']!=-1 or x['totalamountdue']!=-1) else 'pass', axis=1)

    print('Test40===completed===========================================================================')

    transactions_df['Test41'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='credit-transaction' and x['accounttype']=='fd') 
                                                              and (x['smssubtype'] in ('refund-reversal', 'cash-back')) else 'pass', axis=1)

    print('Test41===completed===========================================================================')

    transactions_df['Test42'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='debit-transaction' and x['accounttype']=='credit-card') 
                                                              and (x['balance']!=-1) else 'pass', axis=1)

    print('Test42===completed===========================================================================')

    transactions_df['Test43'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='debit-transaction' and x['accounttype'] in ('bank','debit-card')) 
                                                              and (x['availablelimit']!=-1 ) else 'pass', axis=1)

    print('Test43===completed===========================================================================')

    transactions_df['Test44'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='bill' and x['smssubtype']=='bill-default') 
                                                              and (x['transactiontype']=='auto-debit') else 'pass', axis=1)

    print('Test44===completed===========================================================================')

    transactions_df['Test45'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='debit-transaction' and x['smssubtype']=='cash-withdrawal') 
                                                              and (x['transactionchannel']!='other') else 'pass', axis=1)

    print('Test45===completed===========================================================================')

    transactions_df['Test46'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='debit-transaction' and x['smssubtype']=='cc-bill-payment') 
                                                              and (x['accounttype']=='credit-card') else 'pass', axis=1)

    print('Test46===completed===========================================================================')


    transactions_df['Test47'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='debit-transaction' and x['transactiontype']=='account-transfer') 
                                                              and (x['senderaccountid']!=-1) else 'pass', axis=1)


    print('Test47===completed===========================================================================')

    transactions_df['Test48'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='bank-notification' and x['smssubtype']=='beneficary-add') 
                                                              and (x['sendername']!=-1) else 'pass', axis=1)

    print('Test48===completed===========================================================================')


    transactions_df['Test49'] = transactions_df.apply(lambda x: 'fail' if (re.search('fastag' , x['regex'].lower()) and x['smstype']=='debit-transaction' and x['smssubtype']!='fastag') else 'pass', axis=1)

    print('Test49===completed===========================================================================')
    
    transactions_df['Test50'] = transactions_df.apply(lambda x: 'fail' if (re.search('bill' , x['regex'].lower())) and x['smstype']=='debit-transaction' and (x['transactiontype'] in ('regular','fine','bank-fee','cheque','cash-withdrawal')) else 'pass', axis=1)

    print('Test50===completed===========================================================================')

    transactions_df['Test51'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='bank-notification' and x['smssubtype']=='loan-disbursed'
                                                              and x['amount']!=-1) else 'pass', axis=1)

    print('Test51===completed===========================================================================')

    transactions_df['Test52'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='bill' and (re.search('overdue' , x['regex'].lower()))
                                                              and x['smssubtype']=='bill-payment-confirmation') else 'pass', axis=1)

    print('Test52===completed===========================================================================')

    transactions_df['Test53'] = transactions_df.apply(lambda x: 'fail' if ((re.search('bounced' , x['regex'].lower()))
                                                              and x['transactionchannel']!='auto-debit') else 'pass', axis=1)

    print('Test53===completed===========================================================================')

    transactions_df['Test54'] = transactions_df.apply(lambda x: 'fail' if ((x['transactiontype']=='cash-withdrawal' or x['transactiontype']=='cash')
                                                              and x['transactionchannel']!='other') else 'pass', axis=1)

    print('Test54===completed===========================================================================')

    transactions_df['Test55'] = transactions_df.apply(lambda x: 'fail' if (x['accounttype']=='upi'
                                                              and x['transactionchannel']!='upi') else 'pass', axis=1)

    print('Test55===completed===========================================================================')

    transactions_df['Test56'] = transactions_df.apply(lambda x: 'fail' if (x['smssubtype']=='beneficiary-add'
                                                              and x['accountype']!='bank') else 'pass', axis=1)

    print('Test56===completed===========================================================================')

    transactions_df['Test57'] = transactions_df.apply(lambda x: 'fail' if (re.search('\s*ach\s*|\s*nach\s*|\s*standing\s*instruction|\s*ecs\s*|\s*SI\s*', x['regex'].lower())
                                                              and x['transactionchannel']!='auto-debit') else 'pass', axis=1)

    print('Test57===completed===========================================================================')

    transactions_df['Test58'] = transactions_df.apply(lambda x: 'fail' if (x['smssubtype']=='loan-disbursed'
                                                              and x['accounttype'] not in ('pay-later-account','loan','loan-against-dc','loan-against-cc','credit-line','credit-card','bullet-loan')) else 'pass', axis=1)

    print('Test58===completed===========================================================================')

    transactions_df['Test59'] = transactions_df.apply(lambda x: 'fail' if (x['smssubtype']=='transaction-emi-convert'
                                                              and x['accounttype'] not in ('pay-later-account','loan','loan-against-dc','loan-against-cc','credit-line','credit-card','bullet-loan')) else 'pass', axis=1)

    print('Test59===completed===========================================================================')
    
    transactions_df['Test60'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='debit-transaction' and x['smssubtype']=='transfer-out'
                                                              and x['senderaccountid']!=-1) else 'pass', axis=1)

    print('Test60===completed===========================================================================')

    transactions_df['Test61'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='bill' and x['smssubtype'] in ('bill-reminder','bill-past-due')
                                                              and x['date']!=-1) else 'pass', axis=1)

    print('Test61===completed===========================================================================')

    transactions_df['Test62'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='bill' and x['smssubtype']=='bill-payment-confirmation'
                                                              and x['duedate']!=-1) else 'pass', axis=1)

    print('Test62===completed===========================================================================')
    
    transactions_df['Test63'] = transactions_df.apply(lambda x: 'fail' if (x['transactiontype']=='salary' 
                                                              and x['accounttype']!='bank') else 'pass', axis=1)

    print('Test63===completed===========================================================================')
    
    transactions_df['Test64'] = transactions_df.apply(lambda x: 'fail' if (x['transactiontype']=='cheque' 
                                                              and x['accounttype']!='bank' and x['transactionchannel']!='other') else 'pass', axis=1)

    print('Test64===completed===========================================================================')
    
    transactions_df['Test65'] = transactions_df.apply(lambda x: 'fail' if (x['sms_tags']=='gold-loan' 
                                                              and re.search('gold\s*loan', x['regex'].lower())) else 'pass', axis=1)
    print('Test65===completed===========================================================================')
    
    transactions_df['Test66'] = transactions_df.apply(lambda x: 'fail' if (x['smssubtype']=='balance-notification' and x['accounttype']=='bank' 
                                                              and x['balance']==-1) else 'pass', axis=1)
    print('Test66===completed===========================================================================')

    transactions_df['Test67'] = transactions_df.apply(lambda x: 'fail' if (x['smssubtype'] in ('debit-scheduled','payment-transfer-request') 
                                                              and x['totalamountdue']!=-1 and x['minamountdue']!=-1) else 'pass', axis=1)
    print('Test67===completed===========================================================================')

    transactions_df['Test68'] = transactions_df.apply(lambda x: 'fail' if (x['smssubtype'] in ('credit-card-limit-increased','credit-card-limit-exceeded','credit-card-limit-decreased') 
                                                              and x['accounttype'] not in('credit-card','credit-line','pay-later-account')) else 'pass', axis=1)
    print('Test68===completed===========================================================================')
    
    transactions_df['Test69'] = transactions_df.apply(lambda x: 'fail' if (x['smssubtype']=='balance-notification' and x['accounttype']=='credit-card' 
                                                              and x['availablelimit']==-1) else 'pass', axis=1)
    print('Test69===completed===========================================================================')


    
    
    


    


#     transactions_df.to_excel("sure_regex_checks_output.xlsx")
    return transactions_df



In [7]:
check_transactions_df = sure_regex_checks(check_transactions_df)

Test8===completed===========================================================================
Test12===completed===========================================================================
Test13===completed===========================================================================
Test16===completed===========================================================================
Test17===completed===========================================================================
Test32===completed===========================================================================
Test33===completed===========================================================================
Test34===completed===========================================================================
Test35===completed===========================================================================
Test36===completed===========================================================================
Test37===completed===========================================

In [8]:
check_transactions_df.to_excel('to_check.xlsx',index=0)

### Regex Checks

In [9]:
rule_dict = {}
for col in [x for x in check_transactions_df.columns if 'Test' in x]:
    rule_dict[col]=check_transactions_df[check_transactions_df[col]=='fail'].shape[0]

In [10]:
rule_dict = {}
hashes = []
for col in [x for x in check_transactions_df.columns if 'Test' in x]:
    rule_dict[col]=check_transactions_df[check_transactions_df[col]=='fail'].shape[0]
    for x in check_transactions_df[check_transactions_df[col]=='fail']['hash']:
        hashes.append(x)

In [11]:
hashes

['da2e1010701be0aef62f2deb7c814ca8',
 'b20667521f89c3e667ff43f75d06c15f',
 '97b7c05938b378e59650a6f897c2339d',
 '225a304d8e4aa1237c8dc0de824ec4e1',
 'c1542b591ca7130218b66a7ee5968879',
 '231e1b2dbd31ffbe3afbb98c118d83ce',
 '0582d2cc5d96d8c96fac9dbeba2c8011',
 '34ef04e72653fce3b1b124d030e92812',
 'f1af8773d83da90c3ee4b366a1c9f17a',
 'ddb59401798e56d185bdd13102d900df',
 '06d7c664c567f33c63591aee67d70e71',
 '98d7f76fd01258b62c277d6351d432f4',
 'f597c37d3111640f98d87e7a3f1c2d2b',
 'e2e060332d7f4f3fb53794fee7e54455',
 '3974bc7a4f5bdedf1800c6ebeeb1b0ae',
 '066efe16c69b923b121294656fd42cdf',
 'bd53d7bbd5cc534ca24e9cd52ae0bace',
 '48d314d6569a778c6f61f1bb242e52d6',
 '54951a577eac39bd18870ca73e4f549e',
 '04904184b096d9e7926fb4bcb5eaf092',
 '6ea9e4d587ba33c7cdfabc8b9b4a78bc',
 'a071568e6be3b0c0ca870a70eed8caa7',
 '6e6516f61ea84507aefe7b3480c8fa11',
 '1a2998347bf99d3e9f004ecc9c9c5346',
 'a76daeb56b63959c32457635c9d4d3f9',
 'ef693173c6f1d02a896073a94ecd6f4d',
 'd99cbff52cd06bbfac8edd53ef306500',
 

In [12]:
sorted(rule_dict.items() , key=lambda x:x[1], reverse = True)

[('Test57', 13),
 ('Test17', 8),
 ('Test8', 4),
 ('Test36', 2),
 ('Test42', 2),
 ('Test16', 1),
 ('Test34', 1),
 ('Test39', 1),
 ('Test49', 1),
 ('Test12', 0),
 ('Test13', 0),
 ('Test32', 0),
 ('Test33', 0),
 ('Test35', 0),
 ('Test37', 0),
 ('Test38', 0),
 ('Test40', 0),
 ('Test41', 0),
 ('Test43', 0),
 ('Test44', 0),
 ('Test45', 0),
 ('Test46', 0),
 ('Test47', 0),
 ('Test48', 0),
 ('Test50', 0),
 ('Test51', 0),
 ('Test52', 0),
 ('Test53', 0),
 ('Test54', 0),
 ('Test55', 0),
 ('Test56', 0),
 ('Test58', 0),
 ('Test59', 0),
 ('Test60', 0),
 ('Test61', 0),
 ('Test62', 0),
 ('Test63', 0),
 ('Test64', 0),
 ('Test65', 0),
 ('Test66', 0),
 ('Test67', 0),
 ('Test68', 0),
 ('Test69', 0)]

### Value Checks

In [1]:
# import pandas as pd
# import re
# empty = ['-1','','0','nan','nat','NaT']
# count1 = 0
# count2 = 0
# count3 = 0
# count4 = 0
# count5 = 0
# # amounts list
# amt = ['amount', 'availablelimit', 'balance', 'combinedbalance', 'creditlimit', 'loan_amount', 'minamountdue', 'outstandingamount', 'secondarybalance', 'totalamountdue']
# #emis list
# emi = ['num_emis_due', 'num_emis_paid', 'num_emis_total']
# #dates
# dates = ['date', 'billdate', 'duedate']
# #accounts
# accounts = [ 'pan', 'receiveraccountid', 'secondarypan', 'senderaccountid']
# unwanted_words = ['release', 'payment', 'tollfree','neft','atm', 'imps', 'cash', 'upi','rtgs', 'call','bill', 'bil', 'sms', 'mobile','card', 'block', 'number', 'p2a','salary','transaction','24x7','inft','lpg','subsidy','plan','of', 'at']
# unwanted_words2 = ['release', 'payment','from','tollfree','neft', 'imps', 'cash', 'upi','rtgs', 'call','bill', 'sms','card', 'block', 'number', 'p2a','salary','transaction','24x7','inft','lpg','subsidy','plan','imsi','atm', 'tid']
# trn_note_unwanted  = ['neft','imps', 'cash', 'upi','rtgs','salary','pur']


# def check_trn_note(x):
#     if str(x)=='None' or str(x)=='nan':
#         return 'pass'
#     for each in trn_note_unwanted:
#         if each in x.lower():
#             return 'fail'
#     return 'pass'


# def amount_check(x):
#     global count1
#     flag = 0
#     for col in amt:
#         #print(col)
#         if str(x[col]) not in ['-1','0','','nan','nat','NaT']:
#             if re.search('^[0-9\.\,\-]{1,10}$', str(x[col])):
#                 continue
#             else:
#                 print(x[0],' ',col)
#                 count1 += 1
#                 flag = 1
#                 return True
#     if flag == 0:
#         return False

# def emis_check(x):
#     global count2
#     flag = 0
#     for col in emi:
#         #print(col)z
#         if str(x[col]) not in ['-1','0','','nan','nat','NaT']:
#             if re.search('^[0-9]{1,3}$', str(x[col])):
#                 continue
#             else:
#                 print(x[0],' ',col)
#                 count2 += 1
#                 flag = 1
#                 return True
#     if flag == 0:
#         return False

# def dates_check(x):
#     global count3
#     flag = 0
#     for col in dates:
#         if str(x[col]) not in empty:
#             #print(str(x[col]))
#             if re.search('^\d{2,4}[\.\/\-\s]((([0]?[0-9])|((1)[0-2]))|([A-Za-z]{3}))[\.\/\-\s]([0-2]?[0-9]|(3)[0-1])[,\s]*(([0-1]?\d|2[0-3])?[:]?([0-5]?\d)?[:]?([0-5]?\d)?)?\s*(AM|PM)?$', str(x[col])):
#                 continue
#             elif re.search('^([0-2]?[0-9]|(3)[0-1])[\.\/\-\s]((([0]?[0-9])|((1)[0-2]))|([A-Za-z]{3}))[\.\/\-\s]\d{2,4}[,\s]*(([0-1]?\d|2[0-3])?[:]?([0-5]?\d)?[:]?([0-5]?\d)?)?\s*(AM|PM)?$', str(x[col])):
#                 continue
#             else:
#                 print(x[0],' ',col)
#                 count3 += 1
#                 flag = 1
#                 return True
#     if flag == 0:
#         return False

# def accounts_check(x):
#     global count4
#     flag = 0
#     for col in accounts:
#         if str(x[col]) not in ['-1', '0', '','nan','nat','NaT']:
#             if re.search('^[A-Za-z\*\-\.0-9\@\s\/]{2,50}$', str(x[col])):
#                 continue
#             else:
#                 flag = 1
#                 count4 += 1
#                 return True
#     if flag == 0:
#         return False

# def date_format_check(x):

#     global count5
#     flag = 0
#     if str(x['date']) in empty and str(x['dateformat']) in empty:
#         return False
#     if (str(x['date']) in empty and str(x['dateformat']) not in empty) or (str(x['date']) not in empty and str(x['dateformat']) in empty):
#         count5 += 1
#         return True
#     else:
#         if re.search('^\d{2,4}[\.\/\-\s]((0?[1-9]|1[012])|[A-Za-z]{3})[\.\/\-\s](0?[1-9]|[12][0-9]|3[01])\s*(([0-1]?\d|2[0-3]):([0-5]?\d):([0-5]?\d))?\s*(AM|PM)?$', str(x['date'])):
#             if re.search('^[y]{2,4}[\.\/\-\s][m]{1,3}[\.\/\-\s]([d]{1,2})\s*((HH):(mm):(ss))?$', str(x['dateformat'])):
#                 return False
#         elif re.search('^((0?[1-9]|[12][0-9]|3[01])[\.\/\-\s]((0?[1-9]|1[012])|[A-Za-z]{3})[\.\/\-\s]\d{2,4})\s*(([0-1]?\d|2[0-3]):([0-5]?\d):([0-5]?\d))?\s*(AM|PM)?$', str(x['date'])):
#             if re.search('^[d]{1,2}[\.\/\-\s][m]{1,3}[\.\/\-\s]([y]{2,4})\s*((HH):(mm):(ss))?$', str(x['dateformat'])):
#                 return False
#         else:
#             count5 += 1
#             return True

# def check_unwanted(string, unwanted_words):
#     for i in unwanted_words:
#         if i in string.lower():
#             print(i)
#             return True
#         else:
#             continue
#     return False


# def check_date_len(x):
#     try:
#         if str(x)== 'None' or str(x)=='nan':
#             return 'pass'
#         elif len(x)==1 or len(x)>15:
#             return 'fail'
#         else:
#             return 'pass'
#     except:
#         return 'pass'
        
# def remove_unicode(text):
#     if type(text) == float:
#         return str(text)
#     if text is None:
#         return None
#     return ''.join([i if ord(i) < 128 else ' ' for i in text])
    
    
# def sure_value_checks(transactions_df):
#     print('function starts')
# #     transactions_df = pd.read_excel(extracted_data_csv)
#     print('data loaded')

#     transactions_df['Test5'] = transactions_df.apply(lambda x: 'fail' if x['transactiontype'] == "cash-withdrawal" and (x["amount"] > 40000 or x["amount"]%100 != 0) else 'pass', axis=1)
#     print('Test5===completed===========================================================================')
# #     transactions_df['date'] = transactions_df['date'].apply(lambda x:remove_unicode(x))
#     transactions_df['Test14'] = transactions_df.apply(lambda x: check_date_len(x['duedate']),axis=1)
#     transactions_df['Test15'] = transactions_df.apply(lambda x: 'pass' if (str(x['transactionnote'])=='None' or str(x['transactionnote'])=='nan') else ('fail' if (len(x['transactionnote']) ==1 or  len(x['transactionnote'])>100) else 'pass'),axis=1)
#     transactions_df['Test38'] =  transactions_df.apply(lambda x: check_trn_note(x['transactionnote']),axis=1)
#     # Test27 == Checking Account Formats and Domain
#     transactions_df['Test27'] = transactions_df.apply(lambda x: 'fail' if accounts_check(x) else 'pass', axis = 1)
    
#     transactions_df['pos_len_test'] =transactions_df.pos.apply(lambda x:None if x is None else 'fail' if  len(x)>100 else 'pass')
#     transactions_df['sendername_len_test'] =transactions_df.sendername.apply(lambda x:None if x is None else 'fail' if  len(x)>100 else 'pass')
#     transactions_df['receivername_len_test'] =transactions_df.receivername.apply(lambda x:None if x is None else 'fail' if  len(x)>100 else 'pass')
    
#     print(count4)
#     print('Test27===completed===========================================================================')
#     print('executed successfully')
#     transactions_df.to_excel("sure_value_checks_output2.xlsx")
#     print('data saved')
#     return transactions_df
